# Examen de lab de APR, ETSINF, DSIC, UPV, 17-12-2025

## Apellidos, nombre: DIAGO VALETA, IGNACIO

## Problema (1.25 puntos)


`canada-guesser/Canadian-streetview-cities` es un dataset de imágenes callejeras de $15$ ciudades canadienses, clasificadas por ciudad. Considera los siguientes pasos:

**Paso 0:** $\;$ librerías básicas

In [2]:
import numpy as np; import torch

**Paso 1:** $\;$ lectura de los splits `train[:10%]` y `test[:10%]`

In [3]:
from datasets import load_dataset; ds_name = 'canada-guesser/Canadian-streetview-cities'
[train_ds, test_ds] = load_dataset(ds_name, split=['train[:10%]', 'test[:10%]'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

data/train-00000-of-00027.parquet:   0%|          | 0.00/485M [00:00<?, ?B/s]

...

data/test-00000-of-00003.parquet:   0%|          | 0.00/484M [00:00<?, ?B/s]

data/test-00001-of-00003.parquet:   0%|          | 0.00/488M [00:00<?, ?B/s]

data/test-00002-of-00003.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/135000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [5]:
[train_ds, test_ds]

[Dataset({
     features: ['image', 'label', 'city'],
     num_rows: 13500
 }),
 Dataset({
     features: ['image', 'label', 'city'],
     num_rows: 1500
 })]

**Paso 2:** $\;$ elección de modelo pre-entrenado para hacer fine-tuning

In [6]:
model_checkpoint = "google/vit-base-patch16-224-in21k"

**Paso 3:** $\;$ preproceso de datos

In [7]:
from transformers import AutoImageProcessor
from torchvision.transforms import (CenterCrop, Compose, ToTensor)
image_processor = AutoImageProcessor.from_pretrained(model_checkpoint, use_fast=True)
transforms = Compose([CenterCrop(image_processor.size["height"]), ToTensor()])
def preprocess(batch):
    batch['image'] = [transforms(img.convert("RGB")) for img in batch['image']]
    return batch
train_ds.set_transform(preprocess); test_ds.set_transform(preprocess)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

**Paso 4:** $\;$ lectura del modelo pre-entrenado

In [8]:
from transformers import AutoModelForImageClassification
labels = train_ds.features['label'].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i; id2label[i] = label
model = AutoModelForImageClassification.from_pretrained(model_checkpoint, label2id=label2id, id2label=id2label)

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Paso 5:** $\;$ examen del modelo pre-entrenado

In [9]:
model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed


<p style="page-break-after:always;"></p>


Se pide:

**1\. (0.50 puntos)** $\;$ Define un modelo `lora_model` tras instanciar `LoraConfig` con:
* Rango de las matrices de adaptación: $\;$ $16$
* Parámetro alfa de escalado: $\;$ $16$
* Módulos a adaptar: $\;$ `"query"` y `"value"`
* Dropout LoRA: $\;$ $0.1$
* Sesgo: `"none"`
* Módulos a entrenar por completo: $\;$ `"classifier"`

**2\. (0.50 puntos)** $\;$ Define una variable `args` instanciando `TrainingArguments` con:

* Nombre del modelo adaptado: `f"{model_name}-finetuned"`
* Estrategia de evaluación y grabación: `"epoch"`
* Learning rate: `5e-5`
* Batch size para entrenamiento y evaluación: `32`
* Pasos de acumulación de gradientes: `4`
* Épocas de entrenamiento: `1`
* Ratio de warmup: `0.1`
* Pasos para logging: `10`
* Carga del mejor modelo al final: `True`
* Métrica para determinar el mejor modelo: `"accuracy"`
* Subir al hub: `False`
* Reportar a: `"none"`

**3\. (0.25 puntos)** $\;$ Completa el código siguiente para ejecutar el fine-tuning LoRA de `model` usando `test_ds` como dataset de evaluación

In [10]:
!pip install evaluate

import evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [11]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"]
)

lora_model = get_peft_model(model, peft_config)

lora_model.print_trainable_parameters()

trainable params: 601,359 || all params: 86,411,550 || trainable%: 0.6959


In [13]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    output_dir=f"{model_name}-finetuned",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to="none",
    remove_unused_columns=False
)

In [15]:
import evaluate
import torch
import numpy as np
from transformers import Trainer

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

def collate_fn(examples):
    pixel_values = torch.stack([example["image"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

trainer = Trainer(
    model=lora_model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
    data_collator=collate_fn
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,2.687700,2.682932,0.129333


TrainOutput(global_step=106, training_loss=2.6962189944285266, metrics={'train_runtime': 313.7098, 'train_samples_per_second': 43.033, 'train_steps_per_second': 0.338, 'total_flos': 1.0535959816704e+18, 'train_loss': 2.6962189944285266, 'epoch': 1.0})